In [1]:
# standard library imports
from ast import literal_eval
import itertools
import time
import re

# third-party imports
import numpy as np
import pandas as pd

# customisations
pd.set_option("max_columns", 100)

In [2]:

# read in downloaded data
raw_steam_data = pd.read_csv('steam_app_data.csv')

# print out number of rows and columns
print('Rows:', raw_steam_data.shape[0])
print('Columns:', raw_steam_data.shape[1])

# view first five rows
raw_steam_data.head()

Rows: 1000
Columns: 39


,type,name,steam_appid,required_age,is_free,controller_support,dlc,detailed_description,about_the_game,short_description,fullgame,supported_languages,header_image,website,pc_requirements,mac_requirements,linux_requirements,legal_notice,drm_notice,ext_user_account_notice,developers,publishers,demos,price_overview,packages,package_groups,platforms,metacritic,reviews,categories,genres,screenshots,movies,recommendations,achievements,release_date,support_info,background,content_descriptors
0,game,SiN Episodes: Emergence,1300,0,False,NaN,NaN,"You are John Blade, commander of HardCorps, an...","You are John Blade, commander of HardCorps, an...","You are John Blade, commander of HardCorps, an...",NaN,"English, Russian, French",https://cdn.akamai.steamstatic.com/steam/apps/...,NaN,{'minimum': '\r\n\t\t\t<p><strong>Minimum:</st...,[],[],NaN,NaN,NaN,['Ritual Entertainment'],['Ritual Entertainment'],NaN,"{'currency': 'USD', 'initial': 999, 'final': 9...",[443649],"[{'name': 'default', 'title': 'Buy SiN Episode...","{'windows': True, 'mac': False, 'linux': False}","{'score': 75, 'url': 'https://www.metacritic.c...",NaN,"[{'id': 2, 'description': 'Single-player'}, {'...","[{'id': '1', 'description': 'Action'}]","[{'id': 0, 'path_thumbnail': 'https://cdn.akam...",NaN,{'total': 445},NaN,"{'coming_soon': False, 'date': 'May 10, 2006'}","{'url': '', 'email': ''}",https://cdn.akamai.steamstatic.com/steam/apps/...,"{'ids': [], 'notes': None}"
1,game,Heretic: Shadow of the Serpent Riders,2390,0,False,NaN,NaN,"<p>In a twisted medieval dimension, undead cre...","<p>In a twisted medieval dimension, undead cre...","In a twisted medieval dimension, undead creatu...",NaN,English,https://cdn.akamai.steamstatic.com/steam/apps/...,NaN,{'minimum': '<p><strong>Minimum: </strong>A 10...,[],[],NaN,NaN,NaN,['Raven Software'],['id Software'],NaN,"{'currency': 'USD', 'initial': 499, 'final': 4...","[435, 439]","[{'name': 'default', 'title': 'Buy Heretic: Sh...","{'windows': True, 'mac': False, 'linux': False}",NaN,NaN,"[{'id': 2, 'description': 'Single-player'}, {'...","[{'id': '1', 'description': 'Action'}]","[{'id': 0, 'path_thumbnail': 'https://cdn.akam...",NaN,{'total': 637},NaN,"{'coming_soon': False, 'date': 'Aug 3, 2007'}","{'url': '', 'email': ''}",https://cdn.akamai.steamstatic.com/steam/apps/...,"{'ids': [], 'notes': None}"
2,game,Bloody Good Time,2450,0,False,NaN,NaN,"<p>Congratulations, you’ve just been cast in y...","<p>Congratulations, you’ve just been cast in y...","Congratulations, you’ve just been cast in your...",NaN,English,https://cdn.akamai.steamstatic.com/steam/apps/...,NaN,{'minimum': '<strong>Minimum</strong>\n\t\t\t\...,[],[],© 2010 Ubisoft Entertainment. All Rights Reser...,NaN,NaN,['Outerlight Ltd.'],['Ubisoft'],NaN,"{'currency': 'USD', 'initial': 499, 'final': 4...",[6530],"[{'name': 'default', 'title': 'Buy Bloody Good...","{'windows': True, 'mac': False, 'linux': False}","{'score': 73, 'url': 'https://www.metacritic.c...",NaN,"[{'id': 2, 'description': 'Single-player'}, {'...","[{'id': '1', 'description': 'Action'}]","[{'id': 0, 'path_thumbnail': 'https://cdn.akam...",NaN,{'total': 790},"{'total': 12, 'highlighted': [{'name': 'First ...","{'coming_soon': False, 'date': 'Oct 29, 2010'}","{'url': '', 'email': ''}",https://cdn.akamai.steamstatic.com/steam/apps/...,"{'ids': [], 'notes': None}"
3,game,RIP - Trilogy™,2540,0,False,NaN,NaN,With the completion of the third title in the ...,With the completion of the third title in the ...,With the completion of the third title in the ...,NaN,English,https://cdn.akamai.steamstatic.com/steam/apps/...,NaN,{'minimum': '<strong>Minimum: </strong>Windows...,[],[],NaN,NaN,NaN,['Elephant Games'],[''],NaN,"{'currency': 'USD', 'initial': 499, 'final': 4...",[346],"[{'name': 'default', 'title': 'Buy RIP - Trilo...","{'windows': True, 'mac': False, 'linux': False}",NaN,<strong>GameTunnel gives RIP 3 a 73%</strong><...,"[{'id': 2, 'description': 'Single-player'}, {'...","[{'id': '4', 'description': 'Casua

In [3]:
null_counts = raw_steam_data.isnull().sum()
null_counts

type                          0
name                          0
steam_appid                   0
required_age                  0
is_free                       0
controller_support          768
dlc                         668
detailed_description          0
about_the_game                0
short_description             0
fullgame                   1000
supported_languages           0
header_image                  0
website                     255
pc_requirements               0
mac_requirements              0
linux_requirements            0
legal_notice                538
drm_notice                  989
ext_user_account_notice     966
developers                    3
publishers                    0
demos                       910
price_overview              262
packages                    239
package_groups                0
platforms                     0
metacritic                  732
reviews                     684
categories                   15
genres                        1
screensh

In [4]:
threshold = raw_steam_data.shape[0] // 2

print('Drop columns with more than {} missing rows'.format(threshold))
print()

drop_rows = raw_steam_data.columns[null_counts > threshold]

print('Columns to drop: {}'.format(list(drop_rows)))

Drop columns with more than 500 missing rows

Columns to drop: ['controller_support', 'dlc', 'fullgame', 'legal_notice', 'drm_notice', 'ext_user_account_notice', 'demos', 'metacritic', 'reviews']


In [5]:
raw_steam_data['type'].value_counts(dropna=False)

game    1000
Name: type, dtype: int64

In [6]:
print('Rows to remove:', raw_steam_data[raw_steam_data['type'].isnull()].shape[0])

# preview rows with missing type data
raw_steam_data[raw_steam_data['type'].isnull()].head(3)


Rows to remove: 0


,type,name,steam_appid,required_age,is_free,controller_support,dlc,detailed_description,about_the_game,short_description,fullgame,supported_languages,header_image,website,pc_requirements,mac_requirements,linux_requirements,legal_notice,drm_notice,ext_user_account_notice,developers,publishers,demos,price_overview,packages,package_groups,platforms,metacritic,reviews,categories,genres,screenshots,movies,recommendations,achievements,release_date,support_info,background,content_descriptors


In [7]:
raw_steam_data[(raw_steam_data['name'].isnull()) | (raw_steam_data['name'] == 'none')]

,type,name,steam_appid,required_age,is_free,controller_support,dlc,detailed_description,about_the_game,short_description,fullgame,supported_languages,header_image,website,pc_requirements,mac_requirements,linux_requirements,legal_notice,drm_notice,ext_user_account_notice,developers,publishers,demos,price_overview,packages,package_groups,platforms,metacritic,reviews,categories,genres,screenshots,movies,recommendations,achievements,release_date,support_info,background,content_descriptors


In [8]:
duplicate_rows = raw_steam_data[raw_steam_data.duplicated()]

print('Duplicate rows to remove:', duplicate_rows.shape[0])

duplicate_rows.head(3)


Duplicate rows to remove: 0


,type,name,steam_appid,required_age,is_free,controller_support,dlc,detailed_description,about_the_game,short_description,fullgame,supported_languages,header_image,website,pc_requirements,mac_requirements,linux_requirements,legal_notice,drm_notice,ext_user_account_notice,developers,publishers,demos,price_overview,packages,package_groups,platforms,metacritic,reviews,categories,genres,screenshots,movies,recommendations,achievements,release_date,support_info,background,content_descriptors


In [9]:
def drop_null_cols(df, thresh=0.5):
    """Drop columns with more than a certain proportion of missing values (Default 50%)."""
    cutoff_count = len(df) * thresh
    
    return df.dropna(thresh=cutoff_count, axis=1)


def process_name_type(df):
    """Remove null values in name and type columns, and remove type column."""
    df = df[df['type'].notnull()]
    
    df = df[df['name'].notnull()]
    df = df[df['name'] != 'none']
    
    df = df.drop('type', axis=1)
    
    return df
    

def process(df):
    """Process data set. Will eventually contain calls to all functions we write."""
    
    # Copy the input dataframe to avoid accidentally modifying original data
    df = df.copy()
    
    # Remove duplicate rows - all appids should be unique
    df = df.drop_duplicates()
    
    # Remove collumns with more than 50% null values
    df = drop_null_cols(df)
    
    # Process rest of columns
    df = process_name_type(df)
    
    return df

print(raw_steam_data.shape)
initial_processing = process(raw_steam_data)
print(initial_processing.shape)
initial_processing.head()

(1000, 39)
(1000, 29)


,name,steam_appid,required_age,is_free,detailed_description,about_the_game,short_description,supported_languages,header_image,website,pc_requirements,mac_requirements,linux_requirements,developers,publishers,price_overview,packages,package_groups,platforms,categories,genres,screenshots,movies,recommendations,achievements,release_date,support_info,background,content_descriptors
0,SiN Episodes: Emergence,1300,0,False,"You are John Blade, commander of HardCorps, an...","You are John Blade, commander of HardCorps, an...","You are John Blade, commander of HardCorps, an...","English, Russian, French",https://cdn.akamai.steamstatic.com/steam/apps/...,NaN,{'minimum': '\r\n\t\t\t<p><strong>Minimum:</st...,[],[],['Ritual Entertainment'],['Ritual Entertainment'],"{'currency': 'USD', 'initial': 999, 'final': 9...",[443649],"[{'name': 'default', 'title': 'Buy SiN Episode...","{'windows': True, 'mac': False, 'linux': False}","[{'id': 2, 'description': 'Single-player'}, {'...","[{'id': '1', 'description': 'Action'}]","[{'id': 0, 'path_thumbnail': 'https://cdn.akam...",NaN,{'total': 445},NaN,"{'coming_soon': False, 'date': 'May 10, 2006'}","{'url': '', 'email': ''}",https://cdn.akamai.steamstatic.com/steam/apps/...,"{'ids': [], 'notes': None}"
1,Heretic: Shadow of the Serpent Riders,2390,0,False,"<p>In a twisted medieval dimension, undead cre...","<p>In a twisted medieval dimension, undead cre...","In a twisted medieval dimension, undead creatu...",English,https://cdn.akamai.steamstatic.com/steam/apps/...,NaN,{'minimum': '<p><strong>Minimum: </strong>A 10...,[],[],['Raven Software'],['id Software'],"{'currency': 'USD', 'initial': 499, 'final': 4...","[435, 439]","[{'name': 'default', 'title': 'Buy Heretic: Sh...","{'windows': True, 'mac': False, 'linux': False}","[{'id': 2, 'description': 'Single-player'}, {'...","[{'id': '1', 'description': 'Action'}]","[{'id': 0, 'path_thumbnail': 'https://cdn.akam...",NaN,{'total': 637},NaN,"{'coming_soon': False, 'date': 'Aug 3, 2007'}","{'url': '', 'email': ''}",https://cdn.akamai.steamstatic.com/steam/apps/...,"{'ids': [], 'notes': None}"
2,Bloody Good Time,2450,0,False,"<p>Congratulations, you’ve just been cast in y...","<p>Congratulations, you’ve just been cast in y...","Congratulations, you’ve just been cast in your...",English,https://cdn.akamai.steamstatic.com/steam/apps/...,NaN,{'minimum': '<strong>Minimum</strong>\n\t\t\t\...,[],[],['Outerlight Ltd.'],['Ubisoft'],"{'currency': 'USD', 'initial': 499, 'final': 4...",[6530],"[{'name': 'default', 'title': 'Buy Bloody Good...","{'windows': True, 'mac': False, 'linux': False}","[{'id': 2, 'description': 'Single-player'}, {'...","[{'id': '1', 'description': 'Action'}]","[{'id': 0, 'path_thumbnail': 'https://cdn.akam...",NaN,{'total': 790},"{'total': 12, 'highlighted': [{'name': 'First ...","{'coming_soon': False, 'date': 'Oct 29, 2010'}","{'url': '', 'email': ''}",https://cdn.akamai.steamstatic.com/steam/apps/...,"{'ids': [], 'notes': None}"
3,RIP - Trilogy™,2540,0,False,With the completion of the third title in the ...,With the completion of the third title in the ...,With the completion of the third title in the ...,English,https://cdn.akamai.steamstatic.com/steam/apps/...,NaN,{'minimum': '<strong>Minimum: </strong>Windows...,[],[],['Elephant Games'],[''],"{'currency': 'USD', 'initial': 499, 'final': 4...",[346],"[{'name': 'default', 'title': 'Buy RIP - Trilo...","{'windows': True, 'mac': False, 'linux': False}","[{'id': 2, 'description': 'Single-player'}, {'...","[{'id': '4', 'description': 'Casual'}, {'id': ...","[{'id': 0, 'path_thumbnail': 'https://cdn.akam...",NaN,{'total': 234},NaN,"{'coming_soon': False, 'date': 'Jun 1, 2007'}","{'url': '', 'email': ''}",https://cdn.akamai.steamstatic.com/steam/apps/...,"{'ids': [], 'notes': None}"
4,GUN™,2610,17,False,When life robs Colton White of all that matter...,When life robs Colton White of all that matter...,When life robs Colton White of all that matter...,English,https://cdn.akamai.steamstatic.com/st

In [10]:
initial_processing['required_age'].value_counts(dropna=False).sort_index()

0     956
10      1
12      2
13      7
16      3
17     27
18      3
21      1
Name: required_age, dtype: int64

In [11]:
def process_age(df):
    """Format ratings in age column to be in line with the ESRB Age Ratings system."""
    # ESRB Age ratings: 10, 13, 17, 18
    cut_points = [-1, 0, 12, 16, 19, 21]
    label_values = [0, 10, 13, 17, 18]
    
    df['required_age'] = pd.cut(df['required_age'], bins=cut_points, labels=label_values)
    
    return df


age_df = process_age(initial_processing)
age_df['required_age'].value_counts().sort_index()

0     956
10      3
13     10
17     30
18      1
Name: required_age, dtype: int64

In [12]:
age_df['price_overview'].head()

0    {'currency': 'USD', 'initial': 999, 'final': 9...
1    {'currency': 'USD', 'initial': 499, 'final': 4...
2    {'currency': 'USD', 'initial': 499, 'final': 4...
3    {'currency': 'USD', 'initial': 499, 'final': 4...
4    {'currency': 'USD', 'initial': 1999, 'final': ...
Name: price_overview, dtype: object

In [13]:
age_df['platforms'].head()

0    {'windows': True, 'mac': False, 'linux': False}
1    {'windows': True, 'mac': False, 'linux': False}
2    {'windows': True, 'mac': False, 'linux': False}
3    {'windows': True, 'mac': False, 'linux': False}
4    {'windows': True, 'mac': False, 'linux': False}
Name: platforms, dtype: object

In [14]:
platforms_first_row = age_df['platforms'].iloc[0]

print(type(platforms_first_row))

platforms_first_row

<class 'str'>


"{'windows': True, 'mac': False, 'linux': False}"

In [15]:
# Warning It is possible to crash the Python interpreter with a sufficiently large/complex string due to stack depth limitations in Python’s AST compiler.

eval_first_row = literal_eval(platforms_first_row)

print(type(eval_first_row))
print(eval_first_row)

eval_first_row['windows']

<class 'dict'>
{'windows': True, 'mac': False, 'linux': False}


True

In [16]:
age_df['platforms'].isnull().sum()


0

In [17]:
# create string of keys, joined on a semi-colon
';'.join(eval_first_row.keys())

'windows;mac;linux'

In [18]:
platforms = {'windows': True, 'mac': True, 'linux': False}

# list comprehension
print([x for x in platforms.keys() if platforms[x]])

# using list comprehension in join
';'.join(x for x in platforms.keys() if platforms[x])

['windows', 'mac']


'windows;mac'

In [19]:
def process_platforms(df):
    """Split platforms column into separate boolean columns for each platform."""
    # evaluate values in platforms column, so can index into dictionaries
    df = df.copy()
    
    def parse_platforms(x):
        
        d = literal_eval(x)
        
        return ';'.join(platform for platform in d.keys() if d[platform])
    
    df['platforms'] = df['platforms'].apply(parse_platforms)
    
    return df


platforms_df = process_platforms(age_df)
platforms_df['platforms'].value_counts()

windows              612
windows;mac;linux    220
windows;mac          145
windows;linux         23
Name: platforms, dtype: int64

In [20]:
platforms_df['price_overview'].isnull().sum()


262

In [21]:
free_and_null_price = platforms_df[(platforms_df['is_free']) & (platforms_df['price_overview'].isnull())]

free_and_null_price.shape[0]

225

In [22]:
not_free_and_null_price = platforms_df[(platforms_df['is_free'] == False) & (platforms_df['price_overview'].isnull())]

not_free_and_null_price.head()

,name,steam_appid,required_age,is_free,detailed_description,about_the_game,short_description,supported_languages,header_image,website,pc_requirements,mac_requirements,linux_requirements,developers,publishers,price_overview,packages,package_groups,platforms,categories,genres,screenshots,movies,recommendations,achievements,release_date,support_info,background,content_descriptors
40,Sam & Max 301: The Penal Zone,31220,0,False,Wield psychic powers against fearsome foes in ...,Wield psychic powers against fearsome foes in ...,Wield psychic powers against fearsome foes in ...,English<strong>*</strong><br><strong>*</strong...,https://cdn.akamai.steamstatic.com/steam/apps/...,http://store.steampowered.com/app/901399/,[],[],[],['Telltale Games'],['Skunkape Games'],NaN,"[109585, 4172]","[{'name': 'default', 'title': 'Buy Sam & Max 3...",windows,"[{'id': 2, 'description': 'Single-player'}]","[{'id': '1', 'description': 'Action'}, {'id': ...","[{'id': 0, 'path_thumbnail': 'https://cdn.akam...",NaN,{'total': 228},NaN,"{'coming_soon': False, 'date': ''}","{'url': 'https://telltale.com/support/', 'emai...",https://cdn.akamai.steamstatic.com/steam/apps/...,"{'ids': [], 'notes': None}"
41,Iron Grip: Warlord,31700,0,False,Iron Grip: Warlord invigorates tower defense w...,Iron Grip: Warlord invigorates tower defense w...,Iron Grip: Warlord invigorates tower defense w...,English,https://cdn.akamai.steamstatic.com/steam/apps/...,http://igwarlord.com,"{'minimum': '<ul class=""bb_ul""><li><strong>OS:...",[],[],['ISOTX'],['ISOTX'],NaN,NaN,[],windows,"[{'id': 2, 'description': 'Single-player'}, {'...","[{'id': '1', 'description': 'Action'}, {'id': ...","[{'id': 0, 'path_thumbnail': 'https://cdn.akam...",NaN,{'total': 224},"{'total': 18, 'highlighted': [{'name': 'Have G...","{'coming_soon': False, 'date': 'Dec 5, 2008'}","{'url': '', 'email': ''}",https://cdn.akamai.steamstatic.com/steam/apps/...,"{'ids': [], 'notes': None}"
104,Sine Mora,207040,17,False,Sine Mora is a horizontal shoot'em up that pro...,Sine Mora is a horizontal shoot'em up that pro...,Sine Mora is a horizontal shoot'em up that pro...,"English, German, French, Italian, Spanish - Sp...",https://cdn.akamai.steamstatic.com/steam/apps/...,NaN,{'minimum': '<strong>Minimum:</strong><br><ul ...,[],[],['Digital Reality'],['HandyGames'],NaN,[160555],"[{'name': 'default', 'title': 'Buy Sine Mora',...",windows,"[{'id': 2, 'description': 'Single-player'}, {'...","[{'id': '1', 'description': 'Action'}, {'id': ...","[{'id': 0, 'path_thumbnail': 'https://cdn.akam...","[{'id': 2028093, 'name': 'Sine Mora Trailer', ...",{'total': 504},"{'total': 10, 'highlighted': [{'name': 'Nascen...","{'coming_soon': False, 'date': 'Nov 9, 2012'}","{'url': 'https://handy-games.com/contact/', 'e...",https://cdn.akamai.steamstatic.com/steam/apps/...,"{'ids': [], 'notes': None}"
106,Hero Academy,209270,0,False,<h1>Hero Academy 2 Now Available!</h1><p>Hero ...,"<strong>Includes the Council and TF2 teams, 17...",Test your skills in this light-hearted battle ...,"English<strong>*</strong>, German<strong>*</st...",https://cdn.akamai.steamstatic.com/steam/apps/...,http://www.robotentertainment.com/games/heroac...,"{'minimum': 'OS: Windows XP, Windows Vista, Wi...","{'minimum': '<ul class=""bb_ul""><li><strong>OS:...",{'minimum': 'Video Card: Any OpenGL 2.1+ capab...,['Robot Entertainment'],['Robot Entertainment'],NaN,NaN,[],windows;mac;linux,"[{'id': 2, 'description': 'Single-player'}, {'...","[{'id': '23', 'description': 'Indie'}, {'id': ...","[{'id': 0, 'path_thumbnail': 'https://cdn.akam...","[{'id': 81935, 'name': 'Hero Academy Trailer',...",{'total': 453},"{'total': 69, 'highlighted': [{'name': 'Matrix...","{'coming_soon': False, 'date': ''}",{'url': 'http://support.robotentertainment.com...,https://cdn.akamai.steamstatic.com/steam/apps/...,"{'ids': [], 'notes': None}"
128,Teenage Mutant Ninja Turtles™: Out of the Shadows,228560,0,False,Teenage Mutant Ninja Turtles: Out of the Shado...,Teenage Mutant Ninja Turtles: Out 

In [23]:

def print_steam_links(df):
    """Print links to store page for apps in a dataframe."""
    url_base = "https://store.steampowered.com/app/"
    
    for i, row in df.iterrows():
        appid = row['steam_appid']
        name = row['name']
        
        print(name + ':', url_base + str(appid))
        

print_steam_links(not_free_and_null_price[:5])

Sam & Max 301: The Penal Zone: https://store.steampowered.com/app/31220
Iron Grip: Warlord: https://store.steampowered.com/app/31700
Sine Mora: https://store.steampowered.com/app/207040
Hero Academy: https://store.steampowered.com/app/209270
Teenage Mutant Ninja Turtles™: Out of the Shadows: https://store.steampowered.com/app/228560


In [24]:
platforms_df['price_overview'][0]

"{'currency': 'USD', 'initial': 999, 'final': 999, 'discount_percent': 0, 'initial_formatted': '', 'final_formatted': '$9.99'}"

In [25]:
platforms_df['price_overview'][67]

"{'currency': 'USD', 'initial': 499, 'final': 499, 'discount_percent': 0, 'initial_formatted': '', 'final_formatted': '$4.99'}"

In [26]:

def process_price(df):
    df = df.copy()
        
    def parse_price(x):
        if x is not np.nan:
            return literal_eval(x)
        else:
            return {'currency': 'USD', 'initial': -1}
    
    # evaluate as dictionary and set to -1 if missing
    df['price_overview'] = df['price_overview'].apply(parse_price)
    
    # Create columns from currency and initial values
    df['currency'] = df['price_overview'].apply(lambda x: x['currency'])
    df['price'] = df['price_overview'].apply(lambda x: x['initial'])


    # Set price of free games to 0
    df.loc[df['is_free'], 'price'] = 0

      
    # remove rows where price is -1
    df = df[df['price'] != -1]

    #cannot be translated in CSV file
    df = df.drop(index = 999)

    
 

    return df

price_data = process_price(platforms_df)
price_data.head()

,name,steam_appid,required_age,is_free,detailed_description,about_the_game,short_description,supported_languages,header_image,website,pc_requirements,mac_requirements,linux_requirements,developers,publishers,price_overview,packages,package_groups,platforms,categories,genres,screenshots,movies,recommendations,achievements,release_date,support_info,background,content_descriptors,currency,price
0,SiN Episodes: Emergence,1300,0,False,"You are John Blade, commander of HardCorps, an...","You are John Blade, commander of HardCorps, an...","You are John Blade, commander of HardCorps, an...","English, Russian, French",https://cdn.akamai.steamstatic.com/steam/apps/...,NaN,{'minimum': '\r\n\t\t\t<p><strong>Minimum:</st...,[],[],['Ritual Entertainment'],['Ritual Entertainment'],"{'currency': 'USD', 'initial': 999, 'final': 9...",[443649],"[{'name': 'default', 'title': 'Buy SiN Episode...",windows,"[{'id': 2, 'description': 'Single-player'}, {'...","[{'id': '1', 'description': 'Action'}]","[{'id': 0, 'path_thumbnail': 'https://cdn.akam...",NaN,{'total': 445},NaN,"{'coming_soon': False, 'date': 'May 10, 2006'}","{'url': '', 'email': ''}",https://cdn.akamai.steamstatic.com/steam/apps/...,"{'ids': [], 'notes': None}",USD,999
1,Heretic: Shadow of the Serpent Riders,2390,0,False,"<p>In a twisted medieval dimension, undead cre...","<p>In a twisted medieval dimension, undead cre...","In a twisted medieval dimension, undead creatu...",English,https://cdn.akamai.steamstatic.com/steam/apps/...,NaN,{'minimum': '<p><strong>Minimum: </strong>A 10...,[],[],['Raven Software'],['id Software'],"{'currency': 'USD', 'initial': 499, 'final': 4...","[435, 439]","[{'name': 'default', 'title': 'Buy Heretic: Sh...",windows,"[{'id': 2, 'description': 'Single-player'}, {'...","[{'id': '1', 'description': 'Action'}]","[{'id': 0, 'path_thumbnail': 'https://cdn.akam...",NaN,{'total': 637},NaN,"{'coming_soon': False, 'date': 'Aug 3, 2007'}","{'url': '', 'email': ''}",https://cdn.akamai.steamstatic.com/steam/apps/...,"{'ids': [], 'notes': None}",USD,499
2,Bloody Good Time,2450,0,False,"<p>Congratulations, you’ve just been cast in y...","<p>Congratulations, you’ve just been cast in y...","Congratulations, you’ve just been cast in your...",English,https://cdn.akamai.steamstatic.com/steam/apps/...,NaN,{'minimum': '<strong>Minimum</strong>\n\t\t\t\...,[],[],['Outerlight Ltd.'],['Ubisoft'],"{'currency': 'USD', 'initial': 499, 'final': 4...",[6530],"[{'name': 'default', 'title': 'Buy Bloody Good...",windows,"[{'id': 2, 'description': 'Single-player'}, {'...","[{'id': '1', 'description': 'Action'}]","[{'id': 0, 'path_thumbnail': 'https://cdn.akam...",NaN,{'total': 790},"{'total': 12, 'highlighted': [{'name': 'First ...","{'coming_soon': False, 'date': 'Oct 29, 2010'}","{'url': '', 'email': ''}",https://cdn.akamai.steamstatic.com/steam/apps/...,"{'ids': [], 'notes': None}",USD,499
3,RIP - Trilogy™,2540,0,False,With the completion of the third title in the ...,With the completion of the third title in the ...,With the completion of the third title in the ...,English,https://cdn.akamai.steamstatic.com/steam/apps/...,NaN,{'minimum': '<strong>Minimum: </strong>Windows...,[],[],['Elephant Games'],[''],"{'currency': 'USD', 'initial': 499, 'final': 4...",[346],"[{'name': 'default', 'title': 'Buy RIP - Trilo...",windows,"[{'id': 2, 'description': 'Single-player'}, {'...","[{'id': '4', 'description': 'Casual'}, {'id': ...","[{'id': 0, 'path_thumbnail': 'https://cdn.akam...",NaN,{'total': 234},NaN,"{'coming_soon': False, 'date': 'Jun 1, 2007'}","{'url': '', 'email': ''}",https://cdn.akamai.steamstatic.com/steam/apps/...,"{'ids': [], 'notes': None}",USD,499
4,GUN™,2610,17,False,When life robs Colton White of all that matter...,When life robs Colton White of all that matter...,When life robs Colton White of all that matter...,English,https://cdn.akamai.steamstatic.com/steam/apps/...,NaN,{'minimum': '<strong>Minimum</strong> 3D hardw...,[],[],['Neversoft'],['Activision'],"{'currency': 'USD'

In [27]:
price_data[price_data['currency'] != 'USD']

,name,steam_appid,required_age,is_free,detailed_description,about_the_game,short_description,supported_languages,header_image,website,pc_requirements,mac_requirements,linux_requirements,developers,publishers,price_overview,packages,package_groups,platforms,categories,genres,screenshots,movies,recommendations,achievements,release_date,support_info,background,content_descriptors,currency,price
152,Secrets of Rætikon,246680,0,False,<h1>Features of Secrets of Rætikon</h1><p><ul ...,Ruthless bird versus bird action. Wild animals...,Ruthless bird versus bird action. Wild animals...,Anglais,https://cdn.akamai.steamstatic.com/steam/apps/...,http://secrets-of-raetikon.com,{'minimum': '<strong>Minimale\xa0:</strong><br...,{'minimum': '<strong>Minimale\xa0:</strong><br...,{'minimum': '<strong>Minimale\xa0:</strong><br...,['Broken Rules'],['Broken Rules'],"{'currency': 'EUR', 'initial': 999, 'final': 9...","[29476, 40509]","[{'name': 'default', 'title': 'Acheter Secrets...",windows;mac;linux,"[{'id': 2, 'description': 'Solo'}, {'id': 22, ...","[{'id': '1', 'description': 'Action'}, {'id': ...","[{'id': 0, 'path_thumbnail': 'https://cdn.akam...","[{'id': 2031941, 'name': 'Release Trailer', 't...",{'total': 156},"{'total': 19, 'highlighted': [{'name': 'Lynx T...","{'coming_soon': False, 'date': '17 avr. 2014'}",{'url': 'http://brokenrul.es/vanilla/categorie...,https://cdn.akamai.steamstatic.com/steam/apps/...,"{'ids': [], 'notes': None}",EUR,999
343,Urban Empire,352550,0,False,In <strong>Urban Empire</strong> regieren Sie ...,In <strong>Urban Empire</strong> regieren Sie ...,Urban Empire ist als City Ruler der Vorreiter ...,"Englisch<strong>*</strong>, Französisch<strong...",https://cdn.akamai.steamstatic.com/steam/apps/...,NaN,{'minimum': '<strong>Mindestanforderungen:</st...,[],[],['Reborn Games'],['Kalypso Media Digital'],"{'currency': 'EUR', 'initial': 2999, 'final': ...",[61584],"[{'name': 'default', 'title': 'Urban Empire ka...",windows,"[{'id': 2, 'description': 'Einzelspieler'}, {'...","[{'id': '28', 'description': 'Simulation'}, {'...","[{'id': 0, 'path_thumbnail': 'https://cdn.akam...","[{'id': 256667253, 'name': 'Dev Diary #1 DE', ...",{'total': 1853},"{'total': 74, 'highlighted': [{'name': 'Indust...","{'coming_soon': False, 'date': '20. Jan. 2017'}","{'url': 'http://forum.kalypsomedia.com/', 'ema...",https://cdn.akamai.steamstatic.com/steam/apps/...,"{'ids': [], 'notes': None}",EUR,2999


In [28]:
 def process_language(df):

    """Process supported_languages column into a boolean 'is english' column."""
    df = df.copy()
    
    # drop rows with missing language data
    df = df.dropna(subset=['supported_languages'])
    
    df['english'] = df['supported_languages'].apply(lambda x: 1 if 'english' in x.lower() else 0)
    df = df.drop('supported_languages', axis=1)
    
    return df


language_df = process_language(price_data)
language_df[['name', 'english']].head()

,name,english
0,SiN Episodes: Emergence,1
1,Heretic: Shadow of the Serpent Riders,1
2,Bloody Good Time,1
3,RIP - Trilogy™,1
4,GUN™,1


In [29]:
def process_developers_and_publishers(df):
    # remove rows with missing data
    df = df[(df['developers'].notnull()) & (df['publishers'] != "['']")].copy()
    
    for col in ['developers', 'publishers']:
        df[col] = df[col].apply(lambda x: literal_eval(x))
        
        # filter dataframe to rows with lists longer than 1, and store the number of rows
        num_rows = df[df[col].str.len() > 1].shape[0]
        
        print('Rows in {} column with multiple values:'.format(col), num_rows)

process_developers_and_publishers(language_df)

Rows in developers column with multiple values: 70
Rows in publishers column with multiple values: 50


In [30]:
 def process_developers_and_publishers(df):
    """Parse columns as semicolon-separated string."""
    # remove rows with missing data (~ means not)
    df = df[(df['developers'].notnull()) & (df['publishers'] != "['']")].copy()
    df = df[~(df['developers'].str.contains(';')) & ~(df['publishers'].str.contains(';'))]
    df = df[(df['publishers'] != "['NA']") & (df['publishers'] != "['N/A']")]
    
    # create list for each
    df['developer'] = df['developers'].apply(lambda x: ';'.join(literal_eval(x)))
    df['publisher'] = df['publishers'].apply(lambda x: ';'.join(literal_eval(x)))

    df = df.drop(['developers', 'publishers'], axis=1)
    
    return df

dev_pub_df = process_developers_and_publishers(language_df)
dev_pub_df[['name', 'steam_appid', 'developer', 'publisher']].head()

,name,steam_appid,developer,publisher
0,SiN Episodes: Emergence,1300,Ritual Entertainment,Ritual Entertainment
1,Heretic: Shadow of the Serpent Riders,2390,Raven Software,id Software
2,Bloody Good Time,2450,Outerlight Ltd.,Ubisoft
4,GUN™,2610,Neversoft,Activision
5,Sub Command,2920,Sonalysts,Strategy First


In [31]:
print('Categories:\n')
print('Null values:', dev_pub_df['categories'].isnull().sum())
print()
print(dev_pub_df['categories'][0])

print('\nGenres:\n')
print('Null values:', dev_pub_df['genres'].isnull().sum())
print()
print(dev_pub_df['genres'].iloc[0])
print(dev_pub_df['genres'].iloc[930])

Categories:

Null values: 13

[{'id': 2, 'description': 'Single-player'}, {'id': 15, 'description': 'Stats'}]

Genres:

Null values: 0

[{'id': '1', 'description': 'Action'}]
[{'id': '1', 'description': 'Action'}, {'id': '3', 'description': 'RPG'}, {'id': '28', 'description': 'Simulation'}, {'id': '2', 'description': 'Strategy'}]


In [32]:
def process_categories_and_genres(df):
    df = df.copy()
    df = df[(df['categories'].notnull()) & (df['genres'].notnull())]
    
    for col in ['categories', 'genres']:
        df[col] = df[col].apply(lambda x: ';'.join(item['description'] for item in literal_eval(x)))
    
    return df

cat_gen_df = process_categories_and_genres(dev_pub_df)
cat_gen_df[['steam_appid', 'categories', 'genres']].head()

,steam_appid,categories,genres
0,1300,Single-player;Stats,Action
1,2390,Single-player;Multi-player;Steam Cloud,Action
2,2450,Single-player;Multi-player;Steam Achievements,Action
4,2610,Single-player,Action
5,2920,Single-player;Multi-player;Co-op;Includes leve...,Strategy


In [33]:
def process_achievements_and_descriptors(df):

    df = df.drop(['achievements'], axis=1)
    
    return df

In [34]:
def process_descriptions(df):


    df = df.drop(['detailed_description', 'about_the_game', 'short_description'], axis=1)
    
    return df

In [35]:
def process_media(df):
        
    df = df.drop(['header_image', 'screenshots', 'background', 'movies'], axis=1)
    
    return df



In [36]:
def process_website_and_support(df):

    df = df.drop(['website', 'support_info'], axis=1)

    return df

In [37]:
def process_sys_requirements(df):

    df = df.drop(['pc_requirements', 'mac_requirements', 'linux_requirements'], axis=1)

    return df

In [38]:
def process_dlc(df):

    df = df.drop(['dlc'], axis=1)

    return df

In [39]:
def process_legal_notice(df):

    df = df.drop(['legal_notice'], axis=1)

    return df

In [40]:
def process_metacritic_and_recommendations(df):

    df = df.drop(['metacritic', 'recommendations'], axis=1)

    return df

In [41]:
  def process_packages(df):

    df = df.drop(['packages'], axis=1)

    return df

In [42]:
def process(df):
    """Process data set. Will eventually contain calls to all functions we write."""
    
    # Copy the input dataframe to avoid accidentally modifying original data
    df = df.copy()
    
    # Remove duplicate rows - all appids should be unique
    df = df.drop_duplicates()
    
    # Remove collumns with more than 50% null values
    df = drop_null_cols(df)
    
    # Process columns
    df = process_name_type(df)
    df = process_age(df)
    df = process_platforms(df)
    df = process_price(df)
    df = process_language(df)
    df = process_developers_and_publishers(df)
    df = process_categories_and_genres(df)
    df = process_achievements_and_descriptors(df)
    df = process_descriptions(df)
    df = process_media(df)
    df = process_website_and_support(df)
    df = process_sys_requirements(df)
    df = process_dlc(df)
    df = process_legal_notice(df)
    df = process_metacritic_and_recommendations(df)
    df = process_packages(df)

    return df

steam_data = process(raw_steam_data)
steam_data.head()


,name,steam_appid,required_age,is_free,price_overview,package_groups,platforms,categories,genres,recommendations,release_date,content_descriptors,currency,price,english,developer,publisher
0,SiN Episodes: Emergence,1300,0,False,"{'currency': 'USD', 'initial': 999, 'final': 9...","[{'name': 'default', 'title': 'Buy SiN Episode...",windows,Single-player;Stats,Action,{'total': 445},"{'coming_soon': False, 'date': 'May 10, 2006'}","{'ids': [], 'notes': None}",USD,999,1,Ritual Entertainment,Ritual Entertainment
1,Heretic: Shadow of the Serpent Riders,2390,0,False,"{'currency': 'USD', 'initial': 499, 'final': 4...","[{'name': 'default', 'title': 'Buy Heretic: Sh...",windows,Single-player;Multi-player;Steam Cloud,Action,{'total': 637},"{'coming_soon': False, 'date': 'Aug 3, 2007'}","{'ids': [], 'notes': None}",USD,499,1,Raven Software,id Software
2,Bloody Good Time,2450,0,False,"{'currency': 'USD', 'initial': 499, 'final': 4...","[{'name': 'default', 'title': 'Buy Bloody Good...",windows,Single-player;Multi-player;Steam Achievements,Action,{'total': 790},"{'coming_soon': False, 'date': 'Oct 29, 2010'}","{'ids': [], 'notes': None}",USD,499,1,Outerlight Ltd.,Ubisoft
4,GUN™,2610,17,False,"{'currency': 'USD', 'initial': 1999, 'final': ...","[{'name': 'default', 'title': 'Buy GUN™', 'des...",windows,Single-player,Action,{'total': 1059},"{'coming_soon': False, 'date': 'Oct 13, 2006'}","{'ids': [], 'notes': None}",USD,1999,1,Neversoft,Activision
5,Sub Command,2920,0,False,"{'currency': 'USD', 'initial': 999, 'final': 9...","[{'name': 'default', 'title': 'Buy Sub Command...",windows,Single-player;Multi-player;Co-op;Includes leve...,Strategy,NaN,"{'coming_soon': False, 'date': 'Oct 26, 2006'}","{'ids': [], 'notes': None}",USD,999,1,Sonalysts,Strategy First


In [43]:
steam_data.isnull().sum()

name                     0
steam_appid              0
required_age             0
is_free                  0
price_overview           0
package_groups           0
platforms                0
categories               0
genres                   0
recommendations        306
release_date             0
content_descriptors      0
currency                 0
price                    0
english                  0
developer                0
publisher                0
dtype: int64

In [44]:
raw_steam_data.info(verbose=False, memory_usage="deep")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Columns: 39 entries, type to content_descriptors
dtypes: bool(1), float64(1), int64(2), object(35)
memory usage: 18.4 MB


In [45]:
steam_data.info(verbose=False, memory_usage="deep")

<class 'pandas.core.frame.DataFrame'>
Int64Index: 937 entries, 0 to 998
Columns: 17 entries, name to publisher
dtypes: bool(1), category(1), int64(3), object(12)
memory usage: 1.5 MB


In [46]:
steam_data.to_csv('steam_data_clean.csv', index=False)
